# German N-Ball Embeddings
### contributed by [Merve Sahin](https://github.com/Merve40) <br> <br>
      
This notebook is intended for the AI Language Technology lab (Summer Semester 2019) offered at B-IT (Bonn-Aachen International Center for Information Technology). It provides the pre-processing algorithm of german word-senses, which are used as inputs for the n-ball training.

## Table of Contents
* **[Prerequisite](#Prerequisite)**
* **[Setup](#setup)**
* **[The Germanet Package](#the-germanet-package)**
    * [The Node Class](#the-node-class)
    * [The Tree Class](#the-tree-class)
    * [The GermaNet Utility Class](#The-GermaNet-Utility-Class)
    * [Generate Input Files](#Generate-Input-Files)
* **[Experiment 1: Training and evaluating nball embeddings](#Experiment-1:-Training-and-evaluating-nball-embeddings)**
    * [Experiment 1.1: Training nball embeddings](#Experiment-1.1:-Training-nball-embeddings)
    * [Experiment 1.2: Checking whether tree structures are perfectly embedded into word-embeddings](#Experiment-1.2:-Checking-whether-tree-structures-are-perfectly-embedded-into-word-embeddings)
* **[Experiment 2: Observe neighbors of word-sense using nball embeddings](#Experiment-2:-Observe-neighbors-of-word-sense-using-nball-embeddings)**



# Prerequisite
* Install [Mongo DB](https://docs.mongodb.com/manual/installation/#mongodb-community-edition-installation-tutorials)
* Run Mongo DB

# Setup
Installs **pygermanet**, a python package for querying GermaNet, the german WordNet

In [ ]:
! pip install pygermanet

Downloads the git repository for german n-balls and initializes directories.<br> 
*(Word Embedding file takes up to 4.5 GB space)*

In [6]:
%%script bash
git clone https://github.com/Merve40/nball4tree.git
cd nball4tree/data/
unzip GermaNet.zip
unzip mongo_dump_germanet.zip
mongorestore --db germanet dump/germanet/
wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.de.300.vec.gz
gunzip cc.de.300.vec.gz
mv cc.de.300.vec w2v.vec 
cd ..

Available line magics:
%alias  %alias_magic  %autocall  %automagic  %autosave  %bookmark  %cat  %cd  %clear  %colors  %config  %connect_info  %cp  %debug  %dhist  %dirs  %doctest_mode  %ed  %edit  %env  %gui  %hist  %history  %killbgscripts  %ldir  %less  %lf  %lk  %ll  %load  %load_ext  %loadpy  %logoff  %logon  %logstart  %logstate  %logstop  %ls  %lsmagic  %lx  %macro  %magic  %man  %matplotlib  %mkdir  %more  %mv  %notebook  %page  %pastebin  %pdb  %pdef  %pdoc  %pfile  %pinfo  %pinfo2  %popd  %pprint  %precision  %profile  %prun  %psearch  %psource  %pushd  %pwd  %pycat  %pylab  %qtconsole  %quickref  %recall  %rehashx  %reload_ext  %rep  %rerun  %reset  %reset_selective  %rm  %rmdir  %run  %save  %sc  %set_env  %store  %sx  %system  %tb  %time  %timeit  %unalias  %unload_ext  %who  %who_ls  %whos  %xdel  %xmode

Available cell magics:
%%!  %%HTML  %%SVG  %%bash  %%capture  %%debug  %%file  %%html  %%javascript  %%js  %%latex  %%markdown  %%perl  %%prun  %%pypy  %%python  %%python

# The Germanet Package
The following code shows the datastructures and utility classes for constructing a tree of word-senses, as well as a file containing parent-location codes.

In [ ]:
from pygermanet import load_germanet
from xml.sax import make_parser, handler
import os

## The Node Class
Defines the basis of an **n-ary tree** data structure. The chaining of nodes creates a tree. Each node:
* has an index, indicating the number/position of the node amongst it's siblings. The index is used later to construct the **parent-location-code** of word-senses. According to the research paper, an index cannot be 0.
* has a reference to it's parent and children
* has a word-sense

In [ ]:
class Node:
    
    def __init__(self, word, index=1, parent=None):
        self.index = index
        self.word = word
        self.parent = parent
        self.children = []

    def add_child(self, word):
        if self.has_child(word):
            return None

        index = len(self.children)+1
        node = Node(word, index, self)
        self.children.append(node)
        return node

    def has_child(self, word):
        return self.get_child(word) is not None

    def get_child(self, word):
        for child in self.children:
            if child.word == word:
                return child

    def is_leaf(self):
        return not self.children
    
    def get_path(self):
        """
        Retrieves the parent-location-code by tracing back to 
        the root node and recording the index of each parent.
        """
        indices = []
        node = self.parent
        # if current node is root
        if not node:
            return ['1']

        while(node is not None):
            indices.append(str(node.index))
            node = node.parent
        return indices[::-1]

    def get_path_string(self, depth=None):
        path = self.get_path()
        
        if depth is not None:
            length = depth - len(path)
            path += ['0']*length
        
        return self.word + " "+" ".join(path)

## The Tree Class
Initializes a node with a default root. The tree class provides methods to generate a tree file and a parent-location-code file.<br>
The method **add_hypernym_path** adds hypernym paths queried from *pygermanet* one by one to the tree.


In [ ]:
class Tree:
    
    def __init__(self):
        self.root = Node('*root*')
        self.depth = 1
        self.words = set()

    def add_hypernym_path(self, ordered_path, embedded_words, ignore_duplicates):
        """Adds a hypernym path of a word.

        :param ordered_path: ordered list of parents of a word/synset, starting from root
        :param embedded_words: set containing word-embeddings
        :param ignore_duplicates: True if duplicate nodes with different hypernym paths should be ignored, else False
        """

        node = self.root
        current_len = 0
        for synset in ordered_path[1:]:
            current_len += 1
            child = synset.__str__()[7:-1]
            #avoids nodes with multiple word-compositions
            if len(child.split()) > 1:
                break
            elif child.split('.')[0] not in embedded_words:
                break
            elif node.has_child(child):
                node = node.get_child(child)
            elif ignore_duplicates and child in self.words:
                break
            else:
                self.words.add(child)
                node = node.add_child(child)
            
        if current_len > self.depth:
            self.depth = len(ordered_path)

            

    def write_parent_location_code(self, outputfile):
        """Writes parent locations of all words into a file.

        :param outputfile: file to write into
        """

        if os.path.isfile(outputfile):
            return

        def traverse(node, file):
            code = node.get_path_string(self.depth)
            file.write(code+"\n")
            for child in node.children:
                traverse(child, file)

        node = self.root
        with open(outputfile, 'w') as file:
            traverse(node, file)


    def write_tree(self, outputfile):
        """Writes the elements of the tree into a file.
        The structure of the output follows the breadth-first-search approach.

        :param outputfile: the file to write into
        """

        def traverse(node, visited, file):
            code = node.get_path_string()
            if node.is_leaf():
                if not code in visited:
                    file.write(node.word+"\n")
                    visited.add(code)
            else:
                if not code in visited:
                    file.write(node.word +" "+" ".join(map(lambda n: n.word, node.children)))
                    file.write('\n')
                    visited.add(code)
                for child in node.children:
                    traverse(child, visited, file)

        visited = {'0'}
        node = self.root
        with open(outputfile, 'w') as file:
            traverse(node, visited, file)

## The GermaNet Utility Class
This class scans for XML files containing word-senses, parses them to a **tree** datastructure and loads it into memory.

In [ ]:
class GermaNetUtil:
	__source = ''
	__w2vec_file = ''
	__new_w2v_file = ''
	__ignore_duplicates = True
	

	def __init__(self, sourcefolder, wordvec_file, new_w2v_file, ignore_duplicates=True):
		"""
		:param sourcefolder: folder containing all xml files from GermaNet
		:param wordvec_file: file containing german word-embeddings
		"""

		self.__source = sourcefolder
		self.__w2vec_file = wordvec_file
		self.__new_w2v_file = new_w2v_file
		self.__ignore_duplicates = ignore_duplicates

	def load_tree(self, outputfile):
		"""Creates a tree and fills it with words and hypernyms.

		:param outputfile: the outputfile to be created containing all words
		:return: complete tree
		"""

		germanet = load_germanet()

		# step 1: extract words from GermaNet
		print("extracting words..")
		words, embedded_words = self.__extract_words(germanet, outputfile)
				
		# step 2: fill tree with hypernym paths
		count = 0
		skipped = 0
		mult_paths = 0
		tree = Tree()
		for word in words:
			synset = germanet.synset(word) 
			if synset is None:
				skipped += 1
				continue
			paths = synset.hypernym_paths

			if len(paths) == 0:
				skipped += 1
				return
			elif len(paths) > 0 and isinstance(paths[0], list): # checks if synset has multiple paths
				mult_paths += 1
				for path in paths:
					count +=1
					tree.add_hypernym_path(path, embedded_words, self.__ignore_duplicates)
			else:
				count+=1
				tree.add_hypernym_path(paths, embedded_words, self.__ignore_duplicates)

		print("number of words added = "+str(len(tree.words)))
		print("number of paths = "+str(count))
		print("number of synsets with multiple paths = "+str(mult_paths))
		print("skipped = "+str(skipped))

		return tree

	def __get_synset_name(self, synset):
		return synset.__str__()[7:-1]

	def __extract_words(self, germanet, outputFile):
		"""Extracts words from xml files.
		Only those contained in the word-vector embedding are kept.

		:param outputFile: destination file
		"""

		dir = self.__source
		wordVecFile = self.__w2vec_file

		# extracts all existing words from the word-embedding file
		def get_vector_words(file):
			words = []
			with open(file, 'r') as f_in:
				for line in f_in:
					words.append(line.split()[0])
			return set(words)
	

		def get_words(dir):
			prefixes = ['adj', 'verben', 'nomen']
			files = [dir+file for file in os.listdir(dir) if file.split('.')[0] in prefixes]

			class ContentH(handler.ContentHandler):
				def __init__(self):
					self.words = []
					self.current_content = ""

				def startElement(self, name, attrs):
					self.current_content = ""

				def characters(self, content):
					if len(content.strip().split()) == 1:
						self.current_content += content.strip()

				def endElement(self, name):
					if name.lower() == "orthform" and self.current_content:
						self.words.append(self.current_content)
						
			words = []
			for file in files:
				parser = make_parser()
				content_handler = ContentH()
				parser.setContentHandler(content_handler)
				parser.parse(file)
				words += content_handler.words
			return set(words)



		# step 1: get all words from word-embedding
		embedded_words = get_vector_words(wordVecFile)
		print("number of word embeddings = "+str(len(embedded_words)))

		# skips step 2 and 3
		if os.path.isfile(outputFile):
			print("skipping step 2 and 3")
			synsets = set()
			with open(outputFile, 'r') as file:
				for line in file:
					synsets.add(line[:-1])
				print("number of words in word embeddings and GermaNet = "+str(len(synsets)))
			return synsets, embedded_words

		# step 2: read words from WordNet
		words = get_words(dir)
		print("number of words in GermaNet = "+str(len(words)))
		# step 3: discard words that are not present in word-embedding
		retained_words = words.intersection(embedded_words)
		print("number of same words in word embeddings and GermaNet = "+str(len(retained_words)))

		synsets = []
		with open(outputFile, 'w') as f:
			for word in retained_words:
				lst = [self.__get_synset_name(ele) for ele in germanet.synsets(word)]
				synsets += lst
				f.write("\n".join(lst))
				f.write('\n')

		return set(synsets), embedded_words

	def create_w2v_file(self, tree):
		"""
		Creates a new word-to-vector file, which contains only those vectors that are also contained in Germanet.

		:param tree: Tree
		"""
		words = set()
		q = [tree.root]
		while len(q) > 0:
			node = q.pop()
			words.add(node.word.split('.')[0])

			for child in node.children:
				q.insert(0, child)

		with open(self.__w2vec_file, 'r') as f, open(self.__new_w2v_file, 'w+') as out:
			for line in f:
				word = line.split()[0]
				if word in words:
					out.write(line)

## Generate Input Files
The following method makes a call to *GermaNetUtil* in order to load and fill the tree. It takes following arguments:
* **dir**: directory containing all XML files of GermaNet
* **file_word_embedding**: the location of the word embedding file ( *data/w2v.vec* )
* **out_dir**: output directory to generate the files into ( *data/* )

In [ ]:
def generate_germanet(dir, file_word_embedding, out_dir):
	if not out_dir.endswith('/'):
		out_dir = out_dir + '/'

	output_w2v = out_dir+'ws_w2v.vec'
	output_words = out_dir+'ws_words.txt'
	output_tree = out_dir+'ws_child.txt'
	output_codes = out_dir+'ws_catcode.txt'

	util = GermaNetUtil(dir, file_word_embedding, output_w2v)

	print("----------- loading tree -----------------------")
	tree = util.load_tree(output_words)

	print("\n----------- writing parent location codes ------")
	tree.write_parent_location_code(output_codes)
	print("created "+output_codes)

	print("\n----------- writing tree -----------------------")
	tree.write_tree(output_tree)
	print("created "+output_tree+'\n')

	words = []
	leafs = []
	with open(output_tree, 'r') as f:
		for line in f.readlines():
			words.append(line[:-1])
			if len(line[:-1].split()) == 1:
				leafs.append(line[:-1])

	codes = []
	with open(output_codes, 'r') as f:
		for line in f.readlines():
			codes.append(line.split()[0])

	print("tree:")
	print("nodes="+str(len(codes)))
	print("leafs="+str(len(leafs)))
	print("max-depth="+str(tree.depth))

	print("\n----------- writing new w2v file ---------------")
	util.create_w2v_file(tree)
	print("created "+output_w2v)


In [ ]:
xml_path = 'data/GermaNet/'
word_embedding = 'data/w2v.vec'
output_dir = 'data/'

generate_germanet(xml_path, word_embedding, output_dir)

# Experiment 1: Training and evaluating nball embeddings
## Experiment 1.1: Training nball embeddings
The training can be performed using the generated german input files.<br> 
The resulting files contain over 50k word-senses.<br> 
The training can take up to 25 hrs on a normal computer.<br><br>

The germanet package produces a reduced word embedding file, in order to avoid loading large files into memory. The new word embedding file (*data/ws_w2v.vec*) should be used as input for the training command shown below:


In [ ]:
! touch data/nball.txt
! python nball.py --train_nball data/nball.txt --w2v data/ws_w2v.vec --ws_child data/ws_child.txt --ws_catcode data/ws_catcode.txt --log data/log.txt &

## Experiment 1.2: Checking whether tree structures are perfectly embedded into word-embeddings

In [ ]:
! python nball.py --zero_energy data/data_out/ --ball data/nball.txt --ws_child data/ws_child.txt &

# Experiment 2: Observe neighbors of word-sense using nball embeddings

In [ ]:
! python nball.py --neighbors handeln.v.2 handeln.v.5 Teil.n.2 Teil.n.3 Teil.n.4 Teil.n.5 Gerät.n.1 Gerät.n.2 --ball data/nball.txt --num 12